<a href="https://colab.research.google.com/github/Sirius-Aabhas/CS69002_9A_18CS60R55/blob/master/DL_Assign.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [75]:
import torch
import pandas as pd
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torch.utils.data as data_utils
import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import time

nltk.download('punkt')
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

SEED = 42

torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
device = torch.device('cuda:0')
device

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


device(type='cuda', index=0)

##Fetching data

In [0]:
from google.colab import files
import io
uploaded = files.upload()

In [0]:
type(uploaded), uploaded.keys(), type(uploaded['Train_20K.csv'])


In [0]:
df = pd.read_csv(io.StringIO(uploaded['Train_20K.csv'].decode('utf-8')), sep='\t')
df.head()

##Fetching data for now

In [76]:
df = pd.read_csv('Train_20K.csv', sep='\t')
df.tail()

,text,label
17994,I was pleasantly surprised by the film. Let's ...,1
17995,you must be seeing my comments over many films...,0
17996,This is one of those movies that they did too ...,1
17997,Anyone notice that Tommy only has 3 facial exp...,0
17998,I remember watching ATTACK when it first came ...,0


In [77]:
df_test = pd.read_csv('Test_5K.csv', sep='\t')
df_test.tail()

,text,label
4995,I don't know how to describe this movie. It's ...,0
4996,I found this movie hilarious. The spoofs on ot...,1
4997,My family and I have viewed this movie often o...,1
4998,"The Shining, you know what's weird about this ...",1
4999,Nobody could like this movie for its merit but...,0


In [78]:
print('Number of Negative movie reviews', len(df[df['label']==0]))
print('Number of Positive movie reviews', len(df[df['label']==1]))

Number of Negative movie reviews 8994
Number of Positive movie reviews 9005


In [79]:
text_reviews = df['text'].astype(str).tolist()
text_labels = df['label'].astype(int)

text_reviews = [x.lower() for x in text_reviews]

filtered_text_reviews = []
for sent in text_reviews:
    sent = sent.translate(str.maketrans('', '', string.punctuation))
    word_tokens = word_tokenize(sent)
    filtered_text_reviews.append(' '.join([w for w in word_tokens if (not w in stop_words and w != 'br')]))
            
text_reviews = filtered_text_reviews
print(text_reviews[0], text_labels[0])
print(len(text_reviews), len(text_labels))

john waters given us genuinely enjoyable film certainly isnt without shocking watersesque moments tamer older culty stuff pink flamingoes pecker harkens back johns early mainstream stage reminds viewer kind humor evident polyester overall really fun comedy great moments 1
17999 17999


In [80]:
data = [(text_reviews[i], text_labels[i])for i in range(len(text_labels))]
data[:5]

[('john waters given us genuinely enjoyable film certainly isnt without shocking watersesque moments tamer older culty stuff pink flamingoes pecker harkens back johns early mainstream stage reminds viewer kind humor evident polyester overall really fun comedy great moments',
  1),
 ('first two seasons comedy series strange werent funny drama element bill mother struggling usual problems life element bit depressing didnt mix well th comedy elements probably dropped soon became one funniest comedy series bbc ever made chemistry bill bens characters funny always many brilliant memorable sketches series christmas specials hilarious real treat christmas show came stop main actor gary olsen playing bill passed away sad brilliant actor films n funny man ripbr underrated show sadly disappeared television screens doesnt repeated often though appear uktv gold repeated bbc one two show brilliant comedy new audience',
  1),
 ('unfortunate mess shiner wanted like overthetop antifilm aspirant fact f

In [0]:
from google.colab import files
import io
uploaded = files.upload()

In [0]:
type(uploaded), uploaded.keys(), type(uploaded['Test_5K.csv'])

In [0]:
df_test = pd.read_csv(io.StringIO(uploaded['Test_5K.csv'].decode('utf-8')), sep='\t')
df_test.head()

In [81]:
print('Number of Negative movie reviews', len(df_test[df_test['label']==0]))
print('Number of Positive movie reviews', len(df_test[df_test['label']==1]))

Number of Negative movie reviews 2482
Number of Positive movie reviews 2518


In [82]:
text_reviews_test = df_test['text'].astype(str).tolist()
text_labels_test = df_test['label'].astype(int)

text_reviews_test = [x.lower() for x in text_reviews_test]

filtered_text_reviews = []
for sent in text_reviews_test:
    sent = sent.translate(str.maketrans('', '', string.punctuation))
    word_tokens = word_tokenize(sent)
    filtered_text_reviews.append(' '.join([w for w in word_tokens if (not w in stop_words and w != 'br')]))
            
text_reviews_test = filtered_text_reviews

print(text_reviews_test[0], text_labels_test[0])
print(len(text_reviews_test), len(text_labels_test))

always inaccurate picture homeless tv told lot lies panhandlers early 1990s made everyone look bad claimed made 100 day 2040 day much closer reality someone drove held sign offering work offered work actually went took work physically ableand would offered 100000 id damned sure invested apt prepaid least 2 years kept bank still left 1020000 nl 12 25 cash games casinos usually always win could win decent bankroll instead win 1000 month playing always minimum buying due wanting risk losing homeless cause didnt wan na risk spending money going broke sometimes 10002000 sock slept outside anyone wanting talk contact sevencard2003 yahoo messengeri admit different homeless people though due fact never drank smoke took drugs im longer homeless govt housing 177 month getting ssi spend time winning online poker mom sunflower diversified worked hard get ssi glad days hiding stage convention center casino night sleeping worrying getting caught security finally tv crew picked theyd lot sooner shame

In [83]:
data_test = [(text_reviews_test[i], text_labels_test[i])for i in range(len(text_labels_test))]
data_test[:5]

[('always inaccurate picture homeless tv told lot lies panhandlers early 1990s made everyone look bad claimed made 100 day 2040 day much closer reality someone drove held sign offering work offered work actually went took work physically ableand would offered 100000 id damned sure invested apt prepaid least 2 years kept bank still left 1020000 nl 12 25 cash games casinos usually always win could win decent bankroll instead win 1000 month playing always minimum buying due wanting risk losing homeless cause didnt wan na risk spending money going broke sometimes 10002000 sock slept outside anyone wanting talk contact sevencard2003 yahoo messengeri admit different homeless people though due fact never drank smoke took drugs im longer homeless govt housing 177 month getting ssi spend time winning online poker mom sunflower diversified worked hard get ssi glad days hiding stage convention center casino night sleeping worrying getting caught security finally tv crew picked theyd lot sooner sh

## Creating Bag Of Word (BOW) representation of sentences.

In [0]:
def generate_word_ids(dataset):
    word_to_ix = {}
    word_cntr = {}
    word_set = set()
    #print(dataset)
    for sent in dataset:
        for word in sent.split():
            if word not in word_cntr:
                word_cntr[word] = 1
            else:
                word_cntr[word] += 1
    
    for word in word_cntr:
        if word_cntr[word] >= 5:
            word_to_ix[word] = len(word_to_ix)
            
    word_to_ix['<UNKNOWN>'] = len(word_to_ix)
        
    return word_to_ix

In [85]:
word_to_ix = generate_word_ids(text_reviews + text_reviews_test)
VOCAB_SIZE = len(word_to_ix)
VOCAB_SIZE

30624

In [0]:
def make_bow_vector(sentence, word_to_ix):
    # create a vector of zeros of vocab size = len(word_to_idx)
    vec = torch.zeros(len(word_to_ix))
    for word in sentence.split():
        if word not in word_to_ix:
            #raise ValueError('Word',word,' not present in the dictionary. Sorry!')
            vec[word_to_ix['<UNKNOWN>']]+=1
        else:
            vec[word_to_ix[word]]+=1
    return vec.view(1, -1)

def make_target(label):
    return torch.LongTensor([label])


In [88]:
t1 = time.time()
features = make_bow_vector(text_reviews[0], word_to_ix).to(device)
for i in range(1,len(text_reviews[:5000])):
    vec = make_bow_vector(text_reviews[i], word_to_ix).to(device)
    features = torch.cat((features,vec)).to(device)
    if i%1000 == 0:
        print(time.time() - t1)
        t1 = time.time()
        print(features.shape)
targets = torch.tensor(text_labels[:5000], dtype=torch.int).to(device)

print(time.time() - t1)
print(features.shape)
print(targets.shape)
train = data_utils.TensorDataset(features, targets)
train_loader = data_utils.DataLoader(train, batch_size=32, shuffle=True)

2.010638475418091
torch.Size([1001, 30624])
3.3657262325286865
torch.Size([2001, 30624])
5.285852909088135
torch.Size([3001, 30624])
7.414031028747559
torch.Size([4001, 30624])
9.475709199905396
torch.Size([5000, 30624])
torch.Size([5000])


In [89]:
features[0].shape

torch.Size([30624])

##Task1

In [0]:
class Model1A(nn.Module):
    def __init__(self, vocab_size):
        super(Model1A, self).__init__()
        self.lin1 = nn.Linear(vocab_size, 50)
        self.lin2 = nn.Linear(50, 2)

    def forward(self, x):
        x = self.lin1(x)
        return F.softmax(self.lin2(x))

In [0]:
class Model1B(nn.Module):
    def __init__(self, vocab_size):
        super(Model1B, self).__init__()
        self.lin1 = nn.Linear(vocab_size, 100)
        self.lin2 = nn.Linear(100, 2)

    def forward(self, x):
        x = self.lin1(x)
        return F.softmax(self.lin2(x))

In [0]:
class Model1C(nn.Module):
    def __init__(self, vocab_size):
        super(Model1C, self).__init__()
        self.lin1 = nn.Linear(vocab_size, 200)
        self.lin2 = nn.Linear(200, 2)

    def forward(self, x):
        x = self.lin1(x)
        return F.softmax(self.lin2(x))

In [0]:
class Model1D(nn.Module):
    def __init__(self, vocab_size):
        super(Model1D, self).__init__()
        self.lin1 = nn.Linear(vocab_size, 500)
        self.lin2 = nn.Linear(500, 2)

    def forward(self, x):
        x = self.lin1(x)
        return F.softmax(self.lin2(x))

In [0]:
model1A = Model1A(VOCAB_SIZE)
model1B = Model1B(VOCAB_SIZE)
model1C = Model1C(VOCAB_SIZE)
model1D = Model1D(VOCAB_SIZE)

In [0]:
for param in model1A.parameters():
    print(param,param.size())

##Task2

In [0]:
class Model2A(nn.Module):
    def __init__(self, vocab_size):
        super(Model2A, self).__init__()
        self.lin1 = nn.Linear(vocab_size, 10)
        self.lin2 = nn.Linear(10, 10)
        self.lin3 = nn.Linear(10, 2)

    def forward(self, x): 
        x = self.lin1(x)
        x = self.lin2(x)
        return F.softmax(self.lin3(x))

In [0]:
class Model2B(nn.Module):
    def __init__(self, vocab_size):
        super(Model2B, self).__init__()
        self.lin1 = nn.Linear(vocab_size, 20)
        self.lin2 = nn.Linear(20, 10)
        self.lin3 = nn.Linear(10, 2)

    def forward(self, x): 
        x = self.lin1(x)
        x = self.lin2(x)
        return F.softmax(self.lin3(x))

In [0]:
class Model2C(nn.Module):
    def __init__(self, vocab_size):
        super(Model2C, self).__init__()
        self.lin1 = nn.Linear(vocab_size, 30)
        self.lin2 = nn.Linear(30, 30)
        self.lin3 = nn.Linear(30, 2)

    def forward(self, x): 
        x = self.lin1(x)
        x = self.lin2(x)
        return F.softmax(self.lin3(x))

In [0]:
class Model2D(nn.Module):
    def __init__(self, vocab_size):
        super(Model2D, self).__init__()
        self.lin1 = nn.Linear(vocab_size, 50)
        self.lin2 = nn.Linear(50, 50)
        self.lin3 = nn.Linear(50, 2)

    def forward(self, x): 
        x = self.lin1(x)
        x = self.lin2(x)
        return F.softmax(self.lin3(x))

In [0]:
class Model2E(nn.Module):
    def __init__(self, vocab_size):
        super(Model2E, self).__init__()
        self.lin1 = nn.Linear(vocab_size, 100)
        self.lin2 = nn.Linear(100, 50)
        self.lin3 = nn.Linear(50, 2)

    def forward(self, x): 
        x = self.lin1(x)
        x = self.lin2(x)
        return F.softmax(self.lin3(x))

In [0]:
model2A = Model2A(VOCAB_SIZE)
model2B = Model2B(VOCAB_SIZE)
model2C = Model2C(VOCAB_SIZE)
model2D = Model2D(VOCAB_SIZE)
model2E = Model2E(VOCAB_SIZE)

##Task3

In [0]:
class Model3A(nn.Module):
    def __init__(self, vocab_size):
        super(Model3A, self).__init__()
        self.lin1 = nn.Linear(vocab_size, 100)
        self.lin2 = nn.Linear(100, 50)
        self.lin3 = nn.Linear(50, 10)
        self.lin4 = nn.Linear(10, 2)

    def forward(self, x): 
        x = self.lin1(x)
        x = self.lin2(x)
        x = self.lin3(x)
        return F.softmax(self.lin4(x))

In [0]:
class Model3B(nn.Module):
    def __init__(self, vocab_size):
        super(Model3B, self).__init__()
        self.lin1 = nn.Linear(vocab_size, 200)
        self.lin2 = nn.Linear(200, 100)
        self.lin3 = nn.Linear(100, 10)
        self.lin4 = nn.Linear(10, 2)

    def forward(self, x): 
        x = self.lin1(x)
        x = self.lin2(x)
        x = self.lin3(x)
        return F.softmax(self.lin4(x))

In [0]:
model3A = Model3A(VOCAB_SIZE)
model3B = Model3B(VOCAB_SIZE)

##Task4

In [0]:
class Model4A(nn.Module):
    def __init__(self, vocab_size):
        super(Model4A, self).__init__()
        self.lin1 = nn.Linear(vocab_size, 30)
        self.lin2 = nn.Linear(30, 20)
        self.drop_layer = nn.Dropout(p=0.4)
        self.lin3 = nn.Linear(20, 10)
        self.lin4 = nn.Linear(10,2)

    def forward(self, x): 
        x = self.lin1(x)
        x = self.lin2(x)
        x = self.drop_layer(x)
        x = self.lin3(x)
        return F.softmax(self.lin4(x))

In [0]:
class Model4B(nn.Module):
    def __init__(self, vocab_size):
        super(Model4B, self).__init__()
        self.lin1 = nn.Linear(vocab_size, 100)
        self.drop_layer = nn.Dropout(p=0.4)
        self.lin2 = nn.Linear(100, 100)
        self.lin3 = nn.Linear(100,2)

    def forward(self, x): 
        x = self.lin1(x)
        x = self.drop_layer(x)
        x = self.lin2(x)
        return F.softmax(self.lin3(x))

In [0]:
class Model4C(nn.Module):
    def __init__(self, vocab_size):
        super(Model4C, self).__init__()
        self.lin1 = nn.Linear(vocab_size, 100)
        self.drop_layer = nn.Dropout(p=0.4)
        self.lin2 = nn.Linear(100, 10)
        self.lin3 = nn.Linear(10,2)

    def forward(self, x): 
        x = self.lin1(x)
        x = self.drop_layer(x)
        x = self.lin2(x)
        return F.softmax(self.lin3(x))

In [0]:
model4A = Model4A(VOCAB_SIZE)
model4B = Model4B(VOCAB_SIZE)
model4C = Model4C(VOCAB_SIZE)

##Task5

In [0]:
class Model5A(nn.Module):
    def __init__(self, vocab_size):
        super(Model5A, self).__init__()
        self.lin1 = nn.Linear(vocab_size, 30)
        self.lin2 = nn.Linear(30, 20)
        self.drop_layer = nn.Dropout(p=0.4)
        self.lin3 = nn.Linear(20, 10)
        self.lin4 = nn.Linear(10,2)

    def forward(self, x): 
        x = F.relu(self.lin1(x))
        x = F.relu(self.lin2(x))
        x = self.drop_layer(x)
        x = F.relu(self.lin3(x))
        return F.softmax(self.lin4(x))

In [0]:
class Model5B(nn.Module):
    def __init__(self, vocab_size):
        super(Model5B, self).__init__()
        self.lin1 = nn.Linear(vocab_size, 30)
        self.lin2 = nn.Linear(30, 20)
        self.drop_layer = nn.Dropout(p=0.4)
        self.lin3 = nn.Linear(20, 10)
        self.lin4 = nn.Linear(10,2)

    def forward(self, x): 
        x = F.tanh(self.lin1(x))
        x = F.tanh(self.lin2(x))
        x = self.drop_layer(x)
        x = F.tanh(self.lin3(x))
        return F.softmax(self.lin4(x))

In [0]:
class Model5C(nn.Module):
    def __init__(self, vocab_size):
        super(Model5C, self).__init__()
        self.lin1 = nn.Linear(vocab_size, 30)
        self.lin2 = nn.Linear(30, 20)
        self.drop_layer = nn.Dropout(p=0.4)
        self.lin3 = nn.Linear(20, 10)
        self.lin4 = nn.Linear(10,2)

    def forward(self, x): 
        x = F.sigmoid(self.lin1(x))
        x = F.sigmoid(self.lin2(x))
        x = self.drop_layer(x)
        x = F.sigmoid(self.lin3(x))
        return F.softmax(self.lin4(x))

In [0]:
model5A = Model5A(VOCAB_SIZE)
model5B = Model5B(VOCAB_SIZE)
model5C = Model5C(VOCAB_SIZE)

In [0]:
for param in model5A.parameters():
    print(param,param.size())

##Task7

In [0]:
class Model7A(nn.Module):
    def __init__(self, vocab_size):
        super(Model7A, self).__init__()
        self.lin1 = nn.Linear(vocab_size, 30)
        self.lin2 = nn.Linear(30, 20)
        self.lin3 = nn.Linear(20, 10)
        self.lin4 = nn.Linear(10,2)

    def forward(self, x): 
        x = F.relu(self.lin1(x))
        x = F.relu(self.lin2(x))
        x = F.relu(self.lin3(x))
        return F.softmax(self.lin4(x))

In [0]:
class Model7B(nn.Module):
    def __init__(self, vocab_size):
        super(Model7B, self).__init__()
        self.lin1 = nn.Linear(vocab_size, 30)
        self.lin2 = nn.Linear(30, 20)
        self.lin3 = nn.Linear(20, 10)
        self.lin4 = nn.Linear(10,2)

    def forward(self, x): 
        x = F.relu(self.lin1(x))
        x = F.relu(self.lin2(x))
        x = F.relu(self.lin3(x))
        return F.softmax(self.lin4(x))

In [0]:
class Model7C(nn.Module):
    def __init__(self, vocab_size):
        super(Model7C, self).__init__()
        self.lin1 = nn.Linear(vocab_size, 30)
        self.lin2 = nn.Linear(30, 20)
        self.lin3 = nn.Linear(20, 10)
        self.lin4 = nn.Linear(10,2)

    def forward(self, x): 
        x = F.relu(self.lin1(x))
        x = F.relu(self.lin2(x))
        x = F.relu(self.lin3(x))
        return F.softmax(self.lin4(x))

In [0]:
model7A = Model7A(VOCAB_SIZE)
model7B = Model7B(VOCAB_SIZE)
model7C = Model7C(VOCAB_SIZE)

## Run code

In [0]:
def make_one_hot(labels):
    ret_list = torch.zeros((len(labels), 2))
    for i, label in enumerate(labels):
        ret_list[i][label] = 1
    return ret_list

In [0]:
def train(model, isGpu):
    if isGpu:
        model = model.to(device)
    # define a loss function and an optimizer
    #loss_function = nn.NLLLoss()
    loss_function = nn.CrossEntropyLoss()
    opt = torch.optim.SGD(model.parameters(), lr = 0.1)
    # the training loop
    for epoch in range(30):
        tic = time.time()
        for batch_idx, (instance, label) in enumerate(train_loader):
            # get the training data
            model.zero_grad()
            label = label.long()
            probs = model(instance) # forward pass
            loss = loss_function(probs, label)
            loss.backward()
            opt.step()
        toc = time.time()
        print('EPOCH: {}, CURRENT LOSS: {}, TIME TAKEN: {}'.format(epoch, loss.data, (toc-tic)))


In [0]:
def train_MSE_Hinge(model, isGpu, loss_function):
    if isGpu:
        model = model.to(device)
        
    opt = torch.optim.SGD(model.parameters(), lr = 0.1)
    # the training loop
    for epoch in range(30):
        tic = time.time()
        for batch_idx, (instance, label) in enumerate(train_loader):
            # get the training data
            model.zero_grad()
            label = label.long()
            probs = model(instance) # forward pass
            label = make_one_hot(label).to(device)
            loss = loss_function(probs, label)
            loss.backward()
            opt.step()
        toc = time.time()
        print('EPOCH: {}, CURRENT LOSS: {}, TIME TAKEN: {}'.format(epoch, loss.data, (toc-tic)))


In [0]:
def validate(model):
    model = model.cpu()
    preds = []
    for instance, label in data_test:
        bow_vec = Variable(make_bow_vector(instance, word_to_ix))
        logprobs = model(bow_vec)
        #print(logprobs)
        pred = np.argmax(logprobs.data.cpu().numpy())
        preds.append(pred)

    print('accuracy: {}'.format(accuracy_score(text_labels_test, preds)))
    print('precision: {}'.format(precision_score(text_labels_test, preds)))
    print('recall: {}'.format(recall_score(text_labels_test, preds)))
    print('f1-score: {}'.format(f1_score(text_labels_test, preds)))


In [0]:
def run_and_save(model): 
    train(model, True)
    validate(model)
    torch.save(model, model._get_name()+'.mdl')

In [0]:
run_and_save(model1B)
run_and_save(model1C)
run_and_save(model1D)
run_and_save(model2A)
run_and_save(model2B)
run_and_save(model2C)
run_and_save(model2D)
run_and_save(model2E)
run_and_save(model3A)
run_and_save(model3B)
run_and_save(model4A)
run_and_save(model4B)
run_and_save(model4C)
run_and_save(model5A)
run_and_save(model5B)
run_and_save(model5C)
run_and_save(model7A)
train_MSE_Hinge(model7B, True, nn.MSELoss())
validate(model7B)
torch.save(model7B, 'Model7B.mdl')
train_MSE_Hinge(model7C, True, nn.HingeEmbeddingLoss())
validate(model7C)
torch.save(model7C, 'Model7C.mdl')

In [0]:
mdl_1a = torch.load('Model7B.mdl')

In [0]:
validate(mdl_1a)